In [ ]:
!pip install paddlepaddle paddleocr opencv-python
!pip install "paddleocr>=2.0.1"
import cv2
import os
import time
from paddleocr import PaddleOCR

def FrameCapture(video_path, output_folder, frame_rate):
    vidObj = cv2.VideoCapture(video_path)
    success, image = vidObj.read()
    count = 0
    frame_count = 0

    while success:
        if frame_count % frame_rate == 0:
            # Get the timestamp of the current frame
            timestamp = vidObj.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds
            timestamp_sec = int(timestamp)
            timestamp_msec = int((timestamp - timestamp_sec) * 1000)
            timestamp_formatted = time.strftime("%H:%M:%S", time.gmtime(timestamp_sec))

            # Save the frame with the timestamp as the filename
            frame_filename = f"frame{timestamp_formatted},{timestamp_msec}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_filename), image)
            count += 1

        success, image = vidObj.read()
        frame_count += 1

def perform_ocr_on_folder(input_folder, output_file):
    # Initialize OCR reader
    ocr = PaddleOCR(use_angle_cls=True, lang='ch')  # Specify the language model (Chinese in this case)

    # Get a list of image files in the folder
    image_files = [file for file in os.listdir(input_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

    # Open the output file in write mode
    with open(output_file, 'w') as file:
        # Process each image in the folder
        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)

            # Read the image
            img = cv2.imread(image_path)

            # Convert the image to RGB format
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Perform OCR on the image
            result = ocr.ocr(img_rgb, cls=True)

            # Write the recognized text and timestamp to the output file
            file.write(f"Timestamp: {image_file}\n")
            for res in result:
                for line in res:
                    line_text = line[-1]
                    file.write(f"{line_text}\n")
            file.write("\n")

if __name__ == '__main__':
    video_path = "/content/goy.mp4"
    output_folder = "frames_output"
    desired_frame_rate = 15  # Specify the desired frame rate here
    input_folder = "/content/frames_output"  # Replace with the path to the input folder containing images
    output_file = "/content/frames_output.txt"  # Replace with the desired path to the output file

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    FrameCapture(video_path, output_folder, desired_frame_rate)
    perform_ocr_on_folder(input_folder, output_file)


In [ ]:
!pip install paddlepaddle paddleocr opencv-python
!pip install "paddleocr>=2.0.1"

In [ ]:
!ls
!rm -r frames_output

drive  frames_output  sample_data


In [ ]:
import cv2
import os
import time
from paddleocr import PaddleOCR
from tqdm import tqdm

def FrameCapture(video_path, output_folder, frame_rate):
    vidObj = cv2.VideoCapture(video_path)
    success, image = vidObj.read()
    count = 0
    frame_count = 0

    total_frames = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    progress_bar = tqdm(total=total_frames, desc='Frame Progress', unit='frame')

    while success:
        if frame_count % frame_rate == 0:
            # Get the timestamp of the current frame
            timestamp = vidObj.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds
            timestamp_sec = int(timestamp)
            timestamp_msec = int((timestamp - timestamp_sec) * 1000)
            timestamp_formatted = time.strftime("%H:%M:%S", time.gmtime(timestamp_sec))

            # Save the frame with the timestamp as the filename
            frame_filename = f"frame{timestamp_formatted},{timestamp_msec}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_filename), image)
            count += 1

        success, image = vidObj.read()
        frame_count += 1
        progress_bar.update(1)

    progress_bar.close()

def perform_ocr_on_folder(input_folder, output_file):
    # Initialize OCR reader
    ocr = PaddleOCR(use_gpu=True,  # Use GPU if available
                    lang='ch',  # Set the language model to Chinese
                    rec_algorithm='SVTR_LCNet',  # Use the SRN text recognition model for better accuracy ERROR: rec_algorithm must in ['CRNN', 'SVTR_LCNet']

                    rec_image_shape='3, 48, 1600',  # Set a higher resolution for recognition
                    rec_batch_num=4,  # Increase batch size for faster processing
                    # rec_char_dict_path='ppocr/utils/ppocr_keys_v1.txt',  # Use a custom character dictionary for recognition
                    det_algorithm='DB',  # Use the DB text detection model
                    det_limit_side_len=2400,  # Increase the image side length for detection
                    det_batch_size=4,  # Increase batch size for faster detection
                    det_db_box_thresh=0.1,  # Lower the detection box threshold for more accurate detection
                    det_db_unclip_ratio=1.0,  # Adjust the unclip ratio for better box regression
                    det_east_score_thresh=0.8  # Increase the score threshold for the EAST text detection
                    )

    # Get a list of image files in the folder
    image_files = [file for file in os.listdir(input_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

    progress_bar = tqdm(total=len(image_files), desc='OCR Progress', unit='image')

    # Open the output file in write mode
    with open(output_file, 'w') as file:
        # Process each image in the folder
        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)

            # Read the image
            img = cv2.imread(image_path)

            # Convert the image to RGB format
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Perform OCR on the image
            result = ocr.ocr(img_rgb, use_gpu=True, use_pdserving=False)

            # Write the recognized text and timestamp to the output file
            file.write(f"Timestamp: {image_file}\n")
            for res in result:
                for line in res:
                    line_text = line[-1]
                    file.write(f"{line_text}\n")
            file.write("\n")

            progress_bar.update(1)

    progress_bar.close()

if __name__ == '__main__':
    video_path = "/content/drive/MyDrive/ocrvid/EP.15.v0.1688420420.480p.mp4"
    output_folder = "frames_output"
    desired_frame_rate = 10  # Specify the desired frame rate here
    input_folder = "/content/frames_output"  # Replace with the path to the input folder containing images
    output_file = "/content/frames_output.txt"  # Replace with the desired path to the output file

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    FrameCapture(video_path, output_folder, desired_frame_rate)
    perform_ocr_on_folder(input_folder, output_file)


Frame Progress: 100%|██████████| 17071/17071 [00:52<00:00, 322.95frame/s]

[2023/07/04 18:03:34] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_limit_side_len=2400, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.1, det_db_unclip_ratio=1.0, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/ch/ch_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=4, max_text_length=25, rec_char_dict_path='/usr/loca


OCR Progress:   0%|          | 0/1707 [00:00<?, ?image/s]

TypeError: ignored

In [ ]:
#no bar and gpu
import cv2
import os
import time
from paddleocr import PaddleOCR
from tqdm import tqdm

def FrameCapture(video_path, output_folder, frame_rate):
    vidObj = cv2.VideoCapture(video_path)
    success, image = vidObj.read()
    count = 0
    frame_count = 0

    total_frames = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    progress_bar = tqdm(total=total_frames, desc='Frame Progress', unit='frame')

    while success:
        if frame_count % frame_rate == 0:
            # Get the timestamp of the current frame
            timestamp = vidObj.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds
            timestamp_sec = int(timestamp)
            timestamp_msec = int((timestamp - timestamp_sec) * 1000)
            timestamp_formatted = time.strftime("%H:%M:%S", time.gmtime(timestamp_sec))

            # Save the frame with the timestamp as the filename
            frame_filename = f"frame{timestamp_formatted},{timestamp_msec}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_filename), image)
            count += 1

        success, image = vidObj.read()
        frame_count += 1
        progress_bar.update(1)

    progress_bar.close()

def perform_ocr_on_folder(input_folder, output_file):
    # Initialize OCR reader
    ocr = PaddleOCR(use_angle_cls=True, lang='ch')  # Specify the language model (Chinese in this case)

    # Get a list of image files in the folder
    image_files = [file for file in os.listdir(input_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

    progress_bar = tqdm(total=len(image_files), desc='OCR Progress', unit='image')

    # Open the output file in write mode
    with open(output_file, 'w') as file:
        # Process each image in the folder
        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)

            # Read the image
            img = cv2.imread(image_path)

            # Convert the image to RGB format
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Perform OCR on the image
            result = ocr.ocr(img_rgb, cls=True)

            # Write the recognized text and timestamp to the output file
            file.write(f"Timestamp: {image_file}\n")
            for res in result:
                for line in res:
                    line_text = line[-1]
                    file.write(f"{line_text}\n")
            file.write("\n")

            progress_bar.update(1)

    progress_bar.close()

if __name__ == '__main__':
    video_path = "/content/drive/MyDrive/ocrvid/EP.15.v0.1688420420.480p.mp4"
    output_folder = "frames_output"
    desired_frame_rate = 10  # Specify the desired frame rate here
    input_folder = "/content/frames_output"  # Replace with the path to the input folder containing images
    output_file = "/content/frames_output.txt"  # Replace with the desired path to the output file

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    FrameCapture(video_path, output_folder, desired_frame_rate)
    perform_ocr_on_folder(input_folder, output_file)


In [ ]:
import cv2
import os
import time
from paddleocr import PaddleOCR

def FrameCapture(video_path, output_folder, frame_rate):
    vidObj = cv2.VideoCapture(video_path)
    success, image = vidObj.read()
    count = 0
    frame_count = 0

    while success:
        if frame_count % frame_rate == 0:
            # Get the timestamp of the current frame
            timestamp = vidObj.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds
            timestamp_sec = int(timestamp)
            timestamp_msec = int((timestamp - timestamp_sec) * 1000)
            timestamp_formatted = time.strftime("%H:%M:%S", time.gmtime(timestamp_sec))

            # Save the frame with the timestamp as the filename
            frame_filename = f"frame{timestamp_formatted},{timestamp_msec}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_filename), image)
            count += 1

        success, image = vidObj.read()
        frame_count += 1

def perform_ocr_on_folder(input_folder, output_file):
    # Initialize OCR reader
    ocr = PaddleOCR(use_angle_cls=True, lang='ch')  # Specify the language model (Chinese in this case)

    # Get a list of image files in the folder
    image_files = [file for file in os.listdir(input_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

    # Open the output file in write mode
    with open(output_file, 'w') as file:
        # Process each image in the folder
        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)

            # Read the image
            img = cv2.imread(image_path)

            # Convert the image to RGB format
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Perform OCR on the image
            result = ocr.ocr(img_rgb, cls=True)

            # Write the recognized text and timestamp to the output file
            file.write(f"Timestamp: {image_file}\n")
            for res in result:
                for line in res:
                    line_text = line[-1]
                    file.write(f"{line_text}\n")
            file.write("\n")

if __name__ == '__main__':
    video_path = "/content/drive/MyDrive/ocrvid/EP.15.v0.1688420420.480p.mp4"
    output_folder = "frames_output"
    desired_frame_rate = 20  # Specify the desired frame rate here
    input_folder = "/content/frames_output"  # Replace with the path to the input folder containing images
    output_file = "/content/frames_output.txt"  # Replace with the desired path to the output file

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    FrameCapture(video_path, output_folder, desired_frame_rate)
    perform_ocr_on_folder(input_folder, output_file)

[2023/07/04 17:50:49] ppocr DEBUG: rec_res num  : 69, elapse : 6.354953765869141
[2023/07/04 17:50:50] ppocr DEBUG: dt_boxes num : 1, elapse : 0.375563383102417
[2023/07/04 17:50:50] ppocr DEBUG: cls num  : 1, elapse : 0.015273094177246094
[2023/07/04 17:50:50] ppocr DEBUG: rec_res num  : 1, elapse : 0.10063338279724121
[2023/07/04 17:50:50] ppocr DEBUG: dt_boxes num : 3, elapse : 0.380570650100708
[2023/07/04 17:50:50] ppocr DEBUG: cls num  : 3, elapse : 0.022077560424804688
[2023/07/04 17:50:51] ppocr DEBUG: rec_res num  : 3, elapse : 0.2781951427459717
[2023/07/04 17:50:51] ppocr DEBUG: dt_boxes num : 0, elapse : 0.38356614112854004
[2023/07/04 17:50:51] ppocr DEBUG: cls num  : 0, elapse : 0
[2023/07/04 17:50:51] ppocr DEBUG: rec_res num  : 0, elapse : 1.430511474609375e-06
[2023/07/04 17:50:52] ppocr DEBUG: dt_boxes num : 1, elapse : 0.5112273693084717
[2023/07/04 17:50:52] ppocr DEBUG: cls num  : 1, elapse : 0.01966571807861328
[2023/07/04 17:50:52] ppocr DEBUG: rec_res num  : 1, 

KeyboardInterrupt: ignored

In [ ]:
import os, sys, re

video_file_path = "/content/EP.15.v0.1688420420.480p.mp4" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -c copy "$outputPath"/"$fileName"-TRIM."$fileExtension"